In [1]:
import sys
sys.path.append('..')
import pandas as pd
from datetime import datetime
import os

from core.models.utils import run_spark_session
pd.set_option('display.max.columns',300)

from core.calculator.storage import ModelDB
from core.calculator.core import BackTestHonestConfig, BackTestHonestEngine
from core.calculator.core import TrainingManager

from core.calculator.deposits import DepositsCalculationType
from core.calculator.deposits import DepositIterativeCalculator

from core.definitions import *

from core.models import DepositModels
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
import sqlalchemy
sqlalchemy.__version__

'1.3.13'

In [3]:
from core.models import ScenarioLoader

In [4]:
spark = run_spark_session('run_bakctest')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/23 15:42:39 WARN yarn.Client: Same path resource file:///opt/cloudera/parcels/AnacondaPy37/jars/spark-tree-plotting-0.2.jar added multiple times to distributed cache.
23/05/23 15:42:54 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [5]:
folder = '../data/trained_models'

sqlite_filepath = os.path.join(folder, 'modeldb_test.bin')

DB_URL = f"sqlite:///{sqlite_filepath}"
model_db = ModelDB(DB_URL)

In [7]:
last_train_dt = datetime(year=2020, month=12, day=31)
steps = 3
horizon = 3

In [8]:
DepositModels.trainers

{'plan_close': <core.models.plan_close.plan_close_model.PlanCloseModelTrainer at 0x7fb305539650>,
 'renewal_model': <core.models.renewal.renewal_model.RenewalModelTrainer at 0x7fb0c0d5bc90>,
 'maturity_structure_mass_r0s0': <core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S0ModelTrainer at 0x7fb144f5dfd0>,
 'maturity_structure_mass_r0s1': <core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S1ModelTrainer at 0x7fb13dce8050>,
 'maturity_structure_mass_r1s0': <core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S0ModelTrainer at 0x7fb13dce8590>,
 'maturity_structure_mass_r1s1': <core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S1ModelTrainer at 0x7fb13dce8ad0>,
 'maturity_structure_priv_r0s0': <core.models.newbusiness.maturity_structure.maturity_structure_priv_model.MaturityStructurePrivR0S0ModelTrainer at 0

In [9]:
ENV_NAME = 'hmelevskoi_env'

os.environ['CC'] = 'x86_64-conda-linux-gnu-gcc'
os.environ['CXX'] = 'x86_64-conda-linux-gnu-g++'
os.environ['PATH'] = os.path.abspath(f'/tmp/envs/{ENV_NAME}/bin') + ':' + os.environ['PATH']

In [10]:
config: BackTestHonestConfig = BackTestHonestConfig(
    first_train_end_dt = last_train_dt,
    horizon = horizon,
    steps = steps,
    trainers = DepositModels.trainers,
    data_loaders = DepositModels.dataloaders,
    calculator_type = DepositIterativeCalculator,
    calc_type = DepositsCalculationType.Deposits.Deposits,
    adapter_types = DepositModels.adapter_types,
    scenario_loader = ScenarioLoader()
)
training_manager = TrainingManager(spark, DepositModels.trainers, folder, model_db)   
engine: BackTestHonestEngine = BackTestHonestEngine(spark, config, training_manager, overwrite_models=False) 

In [11]:
%%time
engine.run_all()

INFO:core:missing models: [('sa_general_avg_balance_mass', 1), ('sa_general_avg_balance_mass', 2), ('sa_general_avg_balance_mass', 3), ('sa_kopilka_avg_balance_mass', 1), ('sa_kopilka_avg_balance_mass', 2), ('sa_kopilka_avg_balance_mass', 3), ('sa_general_avg_balance_priv', 1), ('sa_general_avg_balance_priv', 2), ('sa_general_avg_balance_priv', 3), ('sa_kopilka_avg_balance_priv', 1), ('sa_kopilka_avg_balance_priv', 2), ('sa_kopilka_avg_balance_priv', 3), ('sa_general_avg_balance_vip', 1), ('sa_general_avg_balance_vip', 2), ('sa_general_avg_balance_vip', 3), ('sa_kopilka_avg_balance_vip', 1), ('sa_kopilka_avg_balance_vip', 2), ('sa_kopilka_avg_balance_vip', 3)]
INFO:core:train sa_general_avg_balance_mass 1
INFO:core:train_model sa_general_avg_balance_mass 1 2020-12-31 00:00:00
INFO:orbit:Sampling (PyStan) with chains: 4, cores: 8, temperature: 1.000, warmups (per chain): 1000 and samples(per chain): 250.
INFO:core:saving trained model ../data/trained_models/sa_general_avg_balance_mass_2

Timestamp('2020-12-31 00:00:00')

INFO:orbit:Sampling (PyStan) with chains: 4, cores: 8, temperature: 1.000, warmups (per chain): 1000 and samples(per chain): 250.
INFO:core:saving trained model ../data/trained_models/sa_general_avg_balance_priv_201801_202012.pickle
INFO:core:saving model info sa_general_avg_balance_priv
INFO:core:train sa_general_avg_balance_priv 2
INFO:core:train_model sa_general_avg_balance_priv 2 2021-03-31 00:00:00


Timestamp('2021-03-31 00:00:00')

INFO:orbit:Sampling (PyStan) with chains: 4, cores: 8, temperature: 1.000, warmups (per chain): 1000 and samples(per chain): 250.
INFO:core:saving trained model ../data/trained_models/sa_general_avg_balance_priv_201801_202103.pickle
INFO:core:saving model info sa_general_avg_balance_priv
ERROR:core:(sqlite3.IntegrityError) UNIQUE constraint failed: model_info.name
[SQL: INSERT INTO model_info (name, mart, saved_at, git_url) VALUES (?, ?, ?, ?)]
[parameters: ('sa_general_avg_balance_priv', '', '2023-05-23 15:44:07.108461', '')]
(Background on this error at: http://sqlalche.me/e/gkpj)
Traceback (most recent call last):
  File "/tmp/envs/hmelevskoi_env/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1246, in _execute_context
    cursor, statement, parameters, context
  File "/tmp/envs/hmelevskoi_env/lib/python3.7/site-packages/sqlalchemy/engine/default.py", line 588, in do_execute
    cursor.execute(statement, parameters)
sqlite3.IntegrityError: UNIQUE constraint failed: mode

Timestamp('2021-06-30 00:00:00')

INFO:orbit:Sampling (PyStan) with chains: 4, cores: 8, temperature: 1.000, warmups (per chain): 1000 and samples(per chain): 250.
INFO:core:saving trained model ../data/trained_models/sa_general_avg_balance_priv_201801_202106.pickle
INFO:core:saving model info sa_general_avg_balance_priv
ERROR:core:(sqlite3.IntegrityError) UNIQUE constraint failed: model_info.name
[SQL: INSERT INTO model_info (name, mart, saved_at, git_url) VALUES (?, ?, ?, ?)]
[parameters: ('sa_general_avg_balance_priv', '', '2023-05-23 15:44:12.927902', '')]
(Background on this error at: http://sqlalche.me/e/gkpj)
Traceback (most recent call last):
  File "/tmp/envs/hmelevskoi_env/lib/python3.7/site-packages/sqlalchemy/engine/base.py", line 1246, in _execute_context
    cursor, statement, parameters, context
  File "/tmp/envs/hmelevskoi_env/lib/python3.7/site-packages/sqlalchemy/engine/default.py", line 588, in do_execute
    cursor.execute(statement, parameters)
sqlite3.IntegrityError: UNIQUE constraint failed: mode

CPU times: user 50 s, sys: 13.8 s, total: 1min 3s
Wall time: 3min 53s


In [17]:
agg_res_deposit = []
agg_res_sa = []
for step in range(1, steps+1):
    agg_res_deposit.append(engine._calc_results[step].calculated_data['Deposits']['agg_data'])
    agg_res_sa.append(engine._calc_results[step].calculated_data['SavingAccounts'])
    
agg_forecast_deposit = pd.concat(agg_res_deposit)
agg_forecast_sa = pd.concat(agg_res_sa)

In [20]:
agg_forecast_sa

balance,report_dt,segment,general,kopilka,safe
0,2021-01-31,mass,6.586291e+11,4.563217e+11,2.023074e+11
1,2021-02-28,mass,6.578844e+11,4.563217e+11,2.015627e+11
2,2021-03-31,mass,6.585949e+11,4.563217e+11,2.022731e+11
3,2021-01-31,priv,4.725067e+11,3.001729e+11,1.723338e+11
4,2021-02-28,priv,4.728907e+11,3.001729e+11,1.727178e+11
5,2021-03-31,priv,4.735527e+11,3.001729e+11,1.733797e+11
6,2021-01-31,vip,1.547380e+11,9.677012e+10,5.796792e+10
7,2021-02-28,vip,1.542802e+11,9.677012e+10,5.751010e+10
8,2021-03-31,vip,1.547093e+11,9.677012e+10,5.793918e+10
0,2021-04-30,mass,7.556361e+11,5.644787e+11,1.911574e+11


In [8]:
import pandas as pd
a = {'b': pd.DataFrame()}

if True and a.get('a') is not None and (a['a'].shape[0] == 0):
    print(a['a'].shape)